In [111]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models.ldamulticore import LdaMulticore

from gensim.models import Phrases
from gensim.parsing.preprocessing import preprocess_string, remove_stopwords, strip_numeric, strip_punctuation, strip_short, stem_text
from gensim.test.utils import common_corpus
import matplotlib.pyplot as plt

In [2]:
from ipynb.fs.full.util import load_data, preprocess_for_gensim
X, y = load_data()

C:\Users\Alan\Documents\Projects School\696DS\voya_project_spring21\LDA\util.ipynb:19: FutureWarning: The default value of regex will change from True to False in a future version.
  "    relevant_cols = [\"PERMID\", \"CIK\", \"Ticker\", \"year\", \"FilingDate\", \"company_name\", \"Dividend Payer\", \"DPS growth\", \"DPS cut\", \"zEnvironmental\", \"dEnvironmental\", \"sector\"]\n",


In [3]:
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary

# def preprocess_for_gensim(data):
#     for section in ('item1a_risk', 'item7_mda'):
#         doc_list = data[section]
#         doc_tokenized = [simple_preprocess(doc) for doc in doc_list]
#         dictionary = corpora.Dictionary()
#         bow_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in doc_tokenized]
#         data[section+'_bow'] = bow_corpus
    
#     return data

In [4]:
# X = preprocess_for_gensim(X)
# X.to_csv('../../Files/processed_data_bow.csv')
# X, y = load_data(bow=True)

In [128]:
items = {
    'item1a': X[ (X.year >= 2012) & (X.year <= 2014) ]['item1a_risk'],
    'item7': X[ (X.year >= 2012) & (X.year <= 2014) ]['item7_mda']
}

In [129]:
tokenizer = RegexpTokenizer(r'\w+')
for item in items:
    docs = items[item]
    for idx in range(len(docs)):
        docs.iloc[idx] = docs.iloc[idx].lower()  # Convert to lowercase.
        docs.iloc[idx] = tokenizer.tokenize(docs.iloc[idx])  # Split into words.
        docs.iloc[idx] = docs.iloc[idx][4:] # Remove first 4 words
    
    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 1] for doc in docs]
    
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
    
    # Add bigrams and trigrams to docs (only ones that appear 20 times or more).
#     bigram = Phrases(docs, min_count=20)
#     for idx in range(len(docs)):
#         for token in bigram[docs[idx]]:
#             if '_' in token:
#                 # Token is a bigram, add to document.
#                 docs[idx].append(token)
                
    items[item] = docs

C:\Users\Alan\Documents\Projects School\696DS\voya_project_spring21\venv\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\Alan\Documents\Projects School\696DS\voya_project_spring21\venv\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [130]:
dictionaries = {}
for item in items:
    dictionaries[item] = Dictionary(items[item])

In [131]:
for dictionary in dictionaries.values():
    # Filter out words that occur less than 20 documents, or more than 10% of the documents.
    dictionary.filter_extremes(no_below=20, no_above=0.1)

In [132]:
corpus = {}
for item in items:
    corpus[item] = [dictionaries[item].doc2bow(doc) for doc in items[item]]

In [133]:
for item in items:
    print(item + ':')
    print('\tNumber of unique tokens: %d' % len(dictionaries[item]))
    print('\tNumber of documents: %d' % len(corpus[item]))

item1a:
	Number of unique tokens: 8423
	Number of documents: 7662
item7:
	Number of unique tokens: 10005
	Number of documents: 7662


In [134]:
params = {
    'num_topics': 30,
    'chunksize': 2000,
    'passes': 20,
    'iterations': 400,
    'eval_every': None,
    'alpha': 'symmetric',
    'eta': 'auto'
}

In [146]:
id2word = {}
for item in items:
    temp = dictionaries[item][0] # Initialize id2token mappings
    id2word[item] = dictionaries[item].id2token

In [145]:
# for item in items:
#     _id2word = id2word[item]
#     items[item].append(_id2word[len(_id2word)-1])
# dictionaries['item1a'].id2token
# temp = dictionaries['item1a'][0]
# len(dictionaries['item1a'].id2token)

8423

In [147]:
models = {}
for item in items:
    models[item] = LdaMulticore(
        corpus=corpus[item],
        id2word=id2word[item],
        workers=32,
        **params
    )

In [155]:
# models['item1a'].print_topics(num_topics=5, num_words=5)
# models['item7'].print_topics(num_topics=5, num_words=5)
# print(models['item1a'].show_topics(num_topics=5, num_words=5, formatted=True))

# Look at things your throwing out in filtering
# Look at strange occurences "duke"
# Look at total perecent of words that are made up
# Split by sector

In [96]:
results = []
for doc_idx in range(item1a.shape[0]):
    result_doc = {}
    for item in items:
        lda_model = models[item]
        scores = []
        topics = []
        for index, score in sorted(lda_model[corpus[item][doc_idx]], key=lambda tup: -1*tup[1]):
#             print ("Score: {}\t Topic ID: {} Topic: {}".format(score, index, lda_model.print_topic(index, 10)))
            topics.append(dictionaries[item][index])
            scores.append(score)
        result_doc[item] = (topics, scores)
    results.append(result_doc)

In [150]:
for item in items:
    lda_model = models[item]
    print(f'{item}:')
    print(lda_model.alpha)
    print(lda_model.eta)

item1a:
[0.03333334 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334
 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334
 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334
 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334
 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334]
[0.07455198 0.47855908 0.11533958 ... 0.04148679 0.0340654  0.03422894]
item7:
[0.03333334 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334
 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334
 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334
 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334
 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334]
[0.18083873 0.05202858 0.6822381  ... 0.08863842 0.05057403 0.06306964]


In [108]:
lda_model.eta

array([0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02], dtype=float32)